# Notebook 1 - Trechos - Elton Cardoso do Nascimento
RA 233840

In [ ]:
!pip install torchtext
!pip install 'portalocker>=2.0.0'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import one_hot
from torchtext.datasets import IMDB
from collections import Counter
import torch.nn as nn
import torch.optim as optim

#Added
import time
import string

import numpy as np
import matplotlib.pyplot as plt


# I

## I.1

### I.1.a, I.1.b, I.1.c

In [ ]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 20000

counter = Counter()
counterTarget = Counter() # NEW: counter for target
meanN_Word = 0 # NEW: mean words in a sentence
for (target, line) in list(IMDB(split='train')):
    words = line.split() # NEW: avoid duplication
    counter.update(words) # CHANGED: use words variable

    # NEW: Update target counter
    if target == 1: 
      counterTarget.update("-")
    else:
      counterTarget.update("+")

    meanN_Word += len(words) # NEW: add the sentence lenght

nTrain = len(list(IMDB(split='train'))) # NEW: store size of train dataset
meanN_Word /= nTrain # NEW: compute mean words per sentence

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
vocab_size = len(vocab)

# NEW: print required results
print("Amostras positivas:", counterTarget["+"])
print("Amostras negativas:", counterTarget["-"])
print("Total de amostras:", nTrain)
print("Comprimento médio:", meanN_Word, "palavras")

## I.2

### I.2.a

In [ ]:
print("5 palavras mais frequentes:", most_frequent_words[:5])
print("5 palavras menos frequentes:", most_frequent_words[-5:])

### I.2.c

In [ ]:
def count_unknown(sentence):
  n_unknown = 0
  encoded_sentence = encode_sentence(sentence, vocab)
  for encoded_word in encoded_sentence:
    if encoded_word == 0:
      n_unknown += 1

  return n_unknown

total_unknown = 0

for (target, line) in list(IMDB(split='train')):
  encode_sentence(line, vocab)

  total_unknown += count_unknown(line)

print("Total de tokens desconhecidos:", total_unknown)

## I.3

### I.3.b

In [ ]:
trainDataset = []
counterTarget = Counter()
nTrain = 0
for (target, line) in list(IMDB(split='train')):

  if target == 1 and counterTarget["-"] < 100: 
    trainDataset.append((target, line))

    counterTarget.update("-")
    nTrain += 1
  elif target == 2 and counterTarget["+"] < 100:
    trainDataset.append((target, line))
    
    counterTarget.update("+")
    nTrain += 1

  if nTrain == 200:
    break

In [ ]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 20000

counter = Counter()
counterTarget = Counter()
meanN_Word = 0
for (target, line) in trainDataset: #CHANGED: use new dataset
    words = line.split()
    counter.update(words)

    # NEW: Update target counter
    if target == 1: 
      counterTarget.update("-")
    else:
      counterTarget.update("+")

    meanN_Word += len(words)

nTrain = len(list(IMDB(split='train')))
meanN_Word /= nTrain

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
vocab_size = len(vocab)


print("Amostras positivas:", counterTarget["+"])
print("Amostras negativas:", counterTarget["-"])
print("Total de amostras:", nTrain)
print("Comprimento médio:", meanN_Word, "palavras")

# II

## II.1

### II.1.a

In [ ]:
def print_info(dataset):

  counterTarget = Counter()
  meanN_Word = 0
  nSentence = len(dataset)
  for i in range(nSentence):
    encoding, target = dataset[i]

    if target == 0:
      counterTarget.update("-")
    else:
      counterTarget.update("+")

    meanN_Word += int(encoding.sum())

  meanN_Word /= nSentence

  print("Amostras positivas:", counterTarget["+"])
  print("Amostras negativas:", counterTarget["-"])
  print("Total de amostras:", nSentence)
  print("Média de palavras codificadas:", meanN_Word, "palavras")

print("TREINO")
print_info(train_data)

### II.1.b

(OBS: média de palavras codificadas no vetor está em II.1.a)

In [ ]:
mean_coded_words = 0

for data in train_data:
  input_data, _ = data

  mean_coded_words += input_data.sum()

mean_coded_words /= len(train_data)

mean_coded_words

## II.2

### II.2.a

In [ ]:
run_n_batch = 2

In [ ]:
import time

model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()

    n_batch = 0
    time_foward = 0
    time_backward = 0
    time_loader = 0
    start_for = time.time()
    start_loader = time.time()

    for inputs, targets in train_loader:
        end_loader = time.time()
        time_loader += end_loader - start_loader

        ## FORWARD ------------------------------
        start_forward = time.time()


        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        # OBS: O enunciado indica que o passo forward deve
        # começar antes das duas linhas acima (????)

        logits = model(inputs)
        loss = criterion(logits.squeeze(), targets.float())

        end_forward = time.time()
        ## --------------------------------------
        ## BACKWARD -----------------------------
        start_backward = time.time()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        end_backward = time.time()
        ## -------------------------------------

        time_foward += end_forward - start_forward
        time_backward += end_backward - start_backward

        n_batch += 1

        if n_batch == run_n_batch:
          break

        start_loader = time.time()



    end_for = time.time()

    time_for = (end_for - start_for) /n_batch
    time_foward /= n_batch
    time_backward /= n_batch
    time_loader /= n_batch

    print("Tempo médio em 1 laço:", time_for)
    print("Tempo médio em 1 foward:", time_foward)
    print("Tempo médio em 1 backward:", time_backward)
    print("Tempo médio loader things:", time_loader)
    break

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    print(f'Epoch [{epoch+1}/{num_epochs}], \
            Loss: {loss.item():.4f}, \
            Elapsed Time: {epoch_duration:.2f} sec')

### II.2.c

In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, split, vocab):
        self.data = list(IMDB(split=split))

        self.vocab = vocab

        #####################
        ##HERE
        #####################
        
        # Cache data encoding and target
        self.itens = []
        for idx in range(len(self.data)):
          target, line = self.data[idx]
          target = 1 if target == 1 else 0

          # one-hot encoding
          X = torch.zeros(len(self.vocab) + 1)
          for word in encode_sentence(line, self.vocab):
              X[word] = 1

          self.itens.append((X, torch.tensor(target)))
          

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        #####################
        ##AND HERE
        #####################

        #Only retrieves data
        return self.itens[idx]

## II.3

### II.3.a

In [ ]:
lr_space = []
for i in range(7):
  lr_space.append(10**-i)
  lr_space.append(5*10**-i)

lr_space = np.sort(lr_space)

In [ ]:
import time

def train(model, lr:int):
  model = model.to(device)
  # Define loss and optimizer
  criterion = nn.BCEWithLogitsLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr)

  # Training loop
  num_epochs = 5
  for epoch in range(num_epochs):
      start_time = time.time()  # Start time of the epoch
      model.train()

      for inputs, targets in train_loader:

          inputs = inputs.to(device)
          targets = targets.to(device)

          logits = model(inputs)
          loss = criterion(logits.squeeze(), targets.float())


          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

def evaluate(model):
  model.eval()

  with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        logits = model(inputs)
        predicted = torch.round(torch.sigmoid(logits.squeeze()))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total

  return accuracy



In [ ]:
accuracies = np.zeros_like(lr_space, np.float64)

for i in range(len(lr_space)):
  lr = lr_space[i]

  for _ in range(5):
    model = OneHotMLP(vocab_size)
    train(model, lr)
    accuracy = evaluate(model)

    accuracies[i] += accuracy

accuracies /= 5

In [ ]:
fig = plt.figure()

plt.plot(lr_space, accuracies, "o")

plt.xlabel("Taxa de treino")
plt.ylabel("Acurácia")
plt.title("Impacto da taxa de treino")

plt.xscale("log")

plt.show()

fig.savefig("ImpactoDaTaxaDeTreino.png", facecolor="white", transparent=False)

### II.3.b

In [ ]:
index = np.argmax(accuracies)
l_opt = lr_space[index]

## II.4

### II.4.a

In [ ]:
def clean_text(text:str) -> str:
  text = text.lower()

  for punctuation in string.punctuation:
    text = text.replace(punctuation, "")

  return text

In [ ]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 20000

counter = Counter()
for (target, line) in list(IMDB(split='train')):
    #AQUI↓################################
    line = clean_text(line) 
    #AQUI↑################################

    counter.update(line.split())

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
vocab_size = len(vocab)

In [ ]:
def encode_sentence(sentence, vocab):
  #AQUI↓################################
  sentence = clean_text(sentence)
  #AQUI↑################################

  return [vocab.get(word, 0) for word in sentence.split()] # 0 for OOV

# III

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

accuracy = 0

lr = lr_opt

for _ in range(5):
  model = OneHotMLP(vocab_size)
  train(model, lr)
  accuracy += evaluate(model)

accuracy /= 5

accuracy

## III.2

### III.2.a

In [ ]:
i = 0
for inputs, targets in train_loader:
 i +=1 

print("Iterações:", i)

### III.2.b

In [ ]:
i = 0
for inputs, targets in train_loader:
  i +=1
  last_size = len(inputs)

last_size

### III.2.c

In [ ]:
R = [int(sum(targets))/len(targets) for  _, targets in train_loader]

R_mean = np.mean(R)

R_mean

### III.2.d

In [ ]:
print("Entrada", batch[0].shape, batch[0].dtype)
print("Saída", batch[1].shape, batch[1].dtype)

## III.3

### III.3.a

In [ ]:
for batch_size in [1, 128, 512]:
  # define dataloaders
  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

  model = OneHotMLP(vocab_size)
  model = model.to(device)

  # Define loss and optimizer
  criterion = nn.BCEWithLogitsLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr_opt)

  # Training loop
  num_epochs = 5
  for epoch in range(num_epochs):
    model.train()

    for inputs, targets in train_loader:

      inputs = inputs.to(device)
      targets = targets.to(device)

      logits = model(inputs)

      if batch_size == 1:
        loss = criterion(logits[0], targets.float())
      else:
        loss = criterion(logits.squeeze(), targets.float())


      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch+1 in [1,5]:
      print(f"batch_size {batch_size} | epoch {epoch+1} | loss {loss.item():.4f}")

# IV

## IV.1

### IV.1.a

In [ ]:
model = OneHotMLP(vocab_size)

batch = next(iter(train_loader))
(inputs, targets) = batch

logits  = model(inputs)

print(torch.sigmoid(logits).squeeze())

### IV.1.b

In [ ]:
batch = next(iter(train_loader))
(inputs, targets) = batch

logits  = model(inputs)
predicted = torch.round(torch.sigmoid(logits.squeeze()))
total = targets.size(0)
correct = (predicted == targets).sum().item()

print(f'Accuracy: {100 * correct / total}%')

### IV.1.c

In [ ]:
total = 0
for layer in [model.fc1, model.fc2]:
  print(np.prod(layer.weight.shape))
  print(np.prod(layer.bias.shape))

  total += np.prod(layer.weight.shape) + np.prod(layer.bias.shape)

print(total)

# V

## V.1

### V.1.a

In [ ]:
loss = nn.BCELoss()

output = torch.tensor([0.5, 0.5], dtype=torch.float64)
target = torch.tensor([0, 1], dtype=torch.float64)

print(float(loss(output, target)), np.log(0.5))

### V.1.b

In [ ]:
(input,target) = next(iter(train_loader))

model = OneHotMLP(vocab_size)

output = model(input)

prob = torch.sigmoid(output.squeeze())

L = torch.sum(
    (target*torch.log(prob)) + 
     ((1-target)*torch.log(1-prob))
     )

L *= -1/(len(target))

print(L.item())

## V.1.c

In [ ]:
(input,target) = next(iter(train_loader))

model = OneHotMLP(vocab_size)

logits = model(input)
predicted = torch.sigmoid(logits.squeeze())

loss = nn.BCELoss()

print(loss(predicted, target.float()).item())

### V.1.d

In [ ]:
(input,target) = next(iter(train_loader))

model = OneHotMLP(vocab_size)

logits = model(input)

loss = nn.BCEWithLogitsLoss()

print(loss(logits.squeeze(), target.float()).item())

## V.2

### V.2.a

In [ ]:
import time
model = OneHotMLP(vocab_size)
model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr_opt)

total_loss = 0
n = 0
for inputs, targets in train_loader:
  inputs = inputs.to(device)
  targets = targets.to(device)
  logits = model(inputs)
  loss = criterion(logits.squeeze(), targets.float())
  total_loss += loss.item()
  n+=1

print(f"Total Loss: {total_loss/n:.4f}")
print(f"Loss: {loss.item():.4f}")

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        # Forward pass
        logits = model(inputs)
        loss = criterion(logits.squeeze(), targets.float())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    print(f'Epoch [{epoch+1}/{num_epochs}], \
            Loss: {loss.item():.4f}, \
            Elapsed Time: {epoch_duration:.2f} sec')

# VI

## VI.1

### VI.1.a

In [ ]:
len(test_data)

### VI.1.c

In [ ]:
inputs = inputs.to(device)
targets = targets.to(device)
logits = model(inputs)

predicted = torch.round(torch.sigmoid(logits.squeeze()))
predicted2 = (logits.squeeze() > 0).float()

equal = (predicted != predicted2).sum() == 0
equal = equal.item()

print("É igual?", equal)

In [ ]:
%timeit (logits.squeeze() > 0).float()

In [ ]:
%timeit predicted = torch.round(torch.sigmoid(logits.squeeze()))

In [ ]:
%timeit (logits.squeeze() > 0)

## VI.2

### VI.2.a

In [ ]:
loss = nn.BCELoss()

output = torch.tensor([0.5, 0.5], dtype=torch.float64)
target = torch.tensor([0, 1], dtype=torch.float64)

np.exp(float(loss(output, target)))

### VI.2.b

In [ ]:
loss = nn.CrossEntropyLoss()

N_hist = list(range(1, 10))
ppl_hist = []

for N in N_hist:

  output = torch.ones(N, dtype=torch.float)
  
  #Simétrico+média -> apenas 1 classe é necessário
  target = torch.tensor(0, dtype=torch.int64) 

  l = loss(output, target)
  ppl = torch.exp(l)

  ppl_hist.append(ppl.item())

N_hist = np.array(N_hist)
ppl_hist = np.array(ppl_hist)

print(ppl_hist)
print(1/N_hist)

## VI.3

### VI.3.a

In [ ]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    total_loss = 0

    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        logits = model(inputs)
        predicted = torch.round(torch.sigmoid(logits.squeeze()))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

        total_loss += criterion(logits.squeeze(), targets.float())*targets.size(0)

    total_loss /= total
    PPL = torch.exp(total_loss)


    print(f'Test Accuracy: {100 * correct / total}%')
    print("Perplexity:", PPL.item())